In [ ]:
# 3.0 Predict Text with LLMs VS Chat Models

from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI()
chat = ChatOpenAI()

a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

a,b

In [ ]:
# 3.1 Predict Messages with Chat Model

# import message constructors
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1
)

messages = [
    SystemMessage(content="You are a geography expert. You only reply in Italian."),
    AIMessage(content="Ciao, mi chiamo Paolo."),
    HumanMessage(content="What is the distance btw Mexico and Thailand? Also, what is your name?"),
]

chat.predict_messages(messages)

# 3.2 Prompt Templates
- PromptTemplate
    - Use when predicting text
    - Create a template from string

- ChatPromptTemplate
    - Create a template from messages

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template("What is the distance btw {country_a} and {country_b}")
prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geographic expert. You only reply in {language}."),
    ("ai", "Ciao, Mi chiamo {name}!"),
    ("human", "What is the distance btw {country_a} and {country_b}"),
])

prompt = template.format_messages(
    language="Greek",
    name="Paolo",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)

In [ ]:
# 3.3 OutputParser: LLM의 Text response를 list로 변환

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        list_items = text.strip().split(",")
        # ['Hello','How','are ',' you']
        return list(map(str.strip(), list_items))
        # ['Hello','How','are','you'] map each item with method.
    
p = CommaOutputParser()
p.parse("Hello,How,are , you")

chat = ChatOpenAI(temperature=0.1)
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generatin machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}"),
])

prompt = template.format_messages(
    max_items=10,
    question="What are the colours?",
)

result = chat.predict_messages(prompt)
# AIMessage(content='red, orange ... ')
p.parse(result.content)
# ['red', 'orange' ...]

In [ ]:
# 3.3 LangChain Expression Language (LCEL)
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
p = CommaOutputParser()

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generatin machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}"),
])

chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items": 5,
    "question": "What are the Pokemons?",
})

In [ ]:
# 3.4 Chaining Chains

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

vegeterian_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."), ("human", "{recipe}")
])

veg_chain = vegeterian_prompt | chat

              # 2 Runnable Map       | # 3
final_chain = {"recipe": chef_chain} | veg_chain

# 1
final_chain.invoke({"cuisine":"indian"})